# Issue 12 - Medication Timeline : 
Design a plugin to display the timeline of medication administration for an individual patient, utilizing an R library.

In [ ]:
! pip install plotly

In [ ]:
# Import librairies:

import pandas as pd
import seaborn as sns
import matplotlib as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# NEED TO PUT DRUG EXPOSURE AND DRUG STRENGTH

In [ ]:
drug_exposure = pd.read_csv('Data/drug_exposure.csv')
# drug_exposure.shape #(18229, 23)

#dose_era = pd.read_csv('Data/dose_era.csv')
#dose_era.shape #(117, 7)

#drug_era = pd.read_csv('Data/drug_era.csv')
#drug_era.shape #(7931, 7)

# drugstrength RxNorm -> from Athena
drug_strength = pd.read_csv('Data/DRUG_STRENGTH.csv', sep='\s+')
# drug_strength.shape #(200983, 12)




### INTERESTING COLUMNS:

Index(['drug_exposure_id', PRIMARY KEY
'person_id', PERSON ID

'drug_concept_id', DRUG ID (RxNorm) STANDARD

       'drug_exposure_start_date'_REQUIRED, 'drug_exposure_start_datetime'_WITH HOURS BUT NOT REQUIRED
       'drug_exposure_end_date'_REQUIRED, 'drug_exposure_end_datetime'_WITH HOURS BUT NOT REQUIRED
       
       'drug_type_concept_id', TYPE OF PRESCRIPTION (32838 = EHR Prescription)
       'quantity', _ NO UNIT 
       'route_concept_id',
       'visit_occurrence_id', 
       'drug_source_value', 
       'drug_source_concept_id', NOT STANDARS
       'route_source_value', 
       'dose_unit_source_value' _ UNIT TO WORK WITH : COULD BE TAB, SYRINGE...],
      dtype='object')


## Test 1 patient

In [ ]:
# liste of non_null columns. The others are totaly empty or useless
liste_non_null = ['drug_exposure_id', 'person_id', 'drug_concept_id',  'drug_exposure_start_date', 'drug_exposure_start_datetime',
       'drug_exposure_end_date', 'drug_exposure_end_datetime', 'drug_type_concept_id',   'quantity',
       'route_concept_id', 'visit_occurrence_id', 'drug_source_value', 'drug_source_concept_id', 'route_source_value', 'dose_unit_source_value']

patient1 = drug_exposure.loc[drug_exposure['person_id'] == 3912882389848878631, liste_non_null]



In [ ]:
# Test for dummy graph with 1 drug only:

patient1_drug1 = patient1.loc[patient1.drug_concept_id==40232756,['drug_exposure_start_datetime', 'drug_exposure_end_datetime', 'quantity']].sort_values(by='drug_exposure_start_datetime')
patient1_drug1['cumulative'] = [0.5, 1.5, 3.0, 4.0]

fig = make_subplots(rows = 1, cols = 2, subplot_titles=('not cumulative','cumulative'))
fig.add_trace(go.Scatter(x = patient1_drug1.drug_exposure_start_datetime, y = patient1_drug1.quantity), row=1, col=1)
fig.update_layout(xaxis=dict(rangeslider=dict(autorange=True, range=[patient1_drug1.drug_exposure_start_datetime.min(), patient1_drug1.drug_exposure_start_datetime.max()])))
   
fig.add_trace(go.Scatter(x = patient1_drug1.drug_exposure_start_datetime, y = patient1_drug1.cumulative), row=1, col=2)

fig.show()

# curseur ok, but I can make it if several traces. Only goes to the first one

In [ ]:
# test for dummy graph with all drugs, but on separated panels according ti drug type. 
# won't stay like this since we will transform it into a mg unit for all?

patient1 = patient1.sort_values(by='drug_exposure_start_datetime')
number_types = patient1.dose_unit_source_value.describe()['unique']

fig3 = make_subplots(rows = 1, cols = number_types, subplot_titles=(patient1.dose_unit_source_value.value_counts().index))
colonne = 1

for drug_type in patient1.dose_unit_source_value.value_counts().index:
    #print(drug_type)

    for d in patient1[patient1.dose_unit_source_value == drug_type].drug_concept_id.unique():
        temp = patient1[patient1.drug_concept_id == d]
        fig3.add_trace(go.Scatter(x = temp.drug_exposure_start_datetime, y = temp.quantity, name=f'{d}, {temp.dose_unit_source_value.unique()}'), row=1, col=colonne)

    colonne += 1

fig3.show()



**PLOTLY GRAPH ARE DYNAMIC. DRUGS CAN BE SELECTED OR UNSELECTED IN THE LEGEND. DON'T KNOW ABOUT IN THE PLUG IN THOUGH IF IT IS JUST A SNAPSHOT, IT WON'T WORK.**
SO WE CAN TRY WITH THE BUTTONS AS SUCH:

In [ ]:
# TOGGLE GRAPH - not useful

patient1 = patient1.sort_values(by='drug_exposure_start_datetime')
number_types = patient1.dose_unit_source_value.describe()['unique']

#fig3 = make_subplots(rows = 1, cols = number_types, subplot_titles=(patient1.dose_unit_source_value.value_counts().index))
colonne = 1

for drug_type in patient1.dose_unit_source_value.value_counts().index:
    #print(drug_type)
    traces = []
    buttons = []

    # for each drug type, create a liste of traces and buttons
    for d in patient1[patient1.dose_unit_source_value == drug_type].drug_concept_id.unique():
        temp = patient1[patient1.drug_concept_id == d]

        traces.append(go.Scatter(x = temp.drug_exposure_start_datetime, y = temp.quantity, name=f'{d}, {temp.dose_unit_source_value.unique()}', visible = True))
        
        buttons.append(dict(method='update',
                        label=f'{d}',
                        visible=True,
                        args=[{'visible':[x == d for x in patient1[patient1.dose_unit_source_value == drug_type].drug_concept_id.unique()]}],
                        args2=[{'visible':[x != d for x in patient1[patient1.dose_unit_source_value == drug_type].drug_concept_id.unique()]}]
                        )
                )

    # Make the graph with toggle
    layout = go.Layout(
        updatemenus=[
            dict(
                type='buttons',
                direction='right',
                x=0.7,
                y=1.3,
                showactive=True,
                buttons=buttons
            )
        ],
        showlegend=True

    )

    fig3 = go.Figure(data=traces,layout=layout)
    fig3.update_layout(xaxis=dict(rangeslider=dict(autorange=True, range=[patient1_drug1.drug_exposure_start_datetime.min(), patient1_drug1.drug_exposure_start_datetime.max()])))
   
    fig3.show()
